In [ ]:
! pip install BeautifulSoup4 pandas selenium

In [ ]:
! pip install webdriver-manager


In [13]:
import os

# Dossier 'data' relatif au dossier courant (scraping/)
data_dir = "../data"
subfolders = ["gold", "silver", "bronze"]

for folder in subfolders:
    path = os.path.join(data_dir, folder)
    os.makedirs(path, exist_ok=True)

print("✅ Dossiers créés : ../data/gold, ../data/silver, ../data/bronze")


✅ Dossiers créés : ../data/gold, ../data/silver, ../data/bronze


In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

options = Options()
options.add_argument("--start-maximized")

service = Service("./chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats")
time.sleep(5)

table = driver.find_element(By.ID, "results2024-202591_overall")
rows = table.find_elements(By.TAG_NAME, "tr")[1:]

links = []
names = []

for row in rows:
    try:
        link = row.find_element(By.CSS_SELECTOR, "td[data-stat='team'] a")
        links.append(link.get_attribute("href"))
        names.append(link.text)
    except:
        continue

data_root = "../data/bronze"  # dossier racine pour les équipes
os.makedirs(data_root, exist_ok=True)

all_players = []

for i, link in enumerate(links):
    driver.get(link)
    time.sleep(3)
    team_name = names[i]
    print("📊 Équipe:", team_name)

    # créer un dossier pour chaque équipe
    team_dir = os.path.join(data_root, team_name.replace(" ", "_"))
    os.makedirs(team_dir, exist_ok=True)

    try:
        table = driver.find_element(By.ID, "stats_standard_9")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")

        for row in rows:
            player_name = row.find_element(By.TAG_NAME, "th").text
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) > 0:
                player_data = {
                    "Player": player_name,
                    "Team": team_name,
                    "Nation": cols[1].text,
                    "Pos": cols[2].text,
                    "Age": cols[3].text,
                    "MP": cols[4].text,
                    "Starts": cols[5].text,
                    "Min": cols[6].text,
                    "90s": cols[7].text,
                    "Gls": cols[8].text,
                    "Ast": cols[9].text,
                    "G+A": cols[10].text,
                    "G-PK": cols[11].text,
                    "PK": cols[12].text,
                    "PKatt": cols[13].text,
                    "CrdY": cols[14].text,
                    "CrdR": cols[15].text,
                }
                all_players.append(player_data)

    except Exception as e:
        print(f"⚠️ Tableau introuvable pour {team_name} : {e}")

    # enregistrer les données de cette équipe dans son dossier
    if all_players:
        df = pd.DataFrame(all_players)
        file_path = os.path.join(team_dir, f"{team_name.replace(' ', '_')}_players.csv")
        df.to_csv(file_path, index=False, encoding="utf-8-sig")
        all_players = []  # réinitialiser pour la prochaine équipe

    time.sleep(2)

    

driver.quit()
print("✅ Données enregistrées dans les dossiers des équipes")


📊 Équipe: Liverpool
📊 Équipe: Arsenal
📊 Équipe: Manchester City
📊 Équipe: Chelsea
📊 Équipe: Newcastle Utd
📊 Équipe: Aston Villa
📊 Équipe: Nott'ham Forest
📊 Équipe: Brighton
📊 Équipe: Bournemouth
📊 Équipe: Brentford
📊 Équipe: Fulham
📊 Équipe: Crystal Palace
📊 Équipe: Everton
📊 Équipe: West Ham
📊 Équipe: Manchester Utd
📊 Équipe: Wolves
📊 Équipe: Tottenham
📊 Équipe: Leicester City
📊 Équipe: Ipswich Town
📊 Équipe: Southampton
✅ Données enregistrées dans les dossiers des équipes


## Code pour recuperer les matchs et les joueurs 

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

options = Options()
options.add_argument("--start-maximized")

service = Service("./chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats")
time.sleep(5)

table = driver.find_element(By.ID, "results2024-202591_overall")
rows = table.find_elements(By.TAG_NAME, "tr")[1:]

links = []
names = []

for row in rows:
    try:
        link = row.find_element(By.CSS_SELECTOR, "td[data-stat='team'] a")
        links.append(link.get_attribute("href"))
        names.append(link.text)
    except:
        continue

data_root = "../data/bronze"  # dossier racine
os.makedirs(data_root, exist_ok=True)

for i, link in enumerate(links):
    driver.get(link)
    time.sleep(3)
    team_name = names[i]
    print("📊 Équipe:", team_name)

    # créer un dossier pour chaque équipe
    team_dir = os.path.join(data_root, team_name.replace(" ", "_"))
    os.makedirs(team_dir, exist_ok=True)

    # ------------------- Joueurs -------------------
    all_players = []
    try:
        table_players = driver.find_element(By.ID, "stats_standard_9")
        tbody_players = table_players.find_element(By.TAG_NAME, "tbody")
        rows_players = tbody_players.find_elements(By.TAG_NAME, "tr")

        for row in rows_players:  # ou rows_players pour tous
            player_name = row.find_element(By.TAG_NAME, "th").text
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) > 0:
                player_data = {
                    "Player": player_name,
                    "Team": team_name,
                    "Nation": cols[1].text,
                    "Pos": cols[2].text,
                    "Age": cols[3].text,
                    "MP": cols[4].text,
                    "Starts": cols[5].text,
                    "Min": cols[6].text,
                    "90s": cols[7].text,
                    "Gls": cols[8].text,
                    "Ast": cols[9].text,
                    "G+A": cols[10].text,
                    "G-PK": cols[11].text,
                    "PK": cols[12].text,
                    "PKatt": cols[13].text,
                    "CrdY": cols[14].text,
                    "CrdR": cols[15].text,
                }
                all_players.append(player_data)

        if all_players:
            df_players = pd.DataFrame(all_players)
            file_path_players = os.path.join(team_dir, f"{team_name.replace(' ', '_')}_players.csv")
            df_players.to_csv(file_path_players, index=False, encoding="utf-8-sig")
    except Exception as e:
        print(f"⚠️ Tableau joueurs introuvable pour {team_name} : {e}")

    # ------------------- Matchs -------------------
    all_matches = []
    try:
        table_matches = driver.find_element(By.ID, "div_matchlogs_for")  # ID du tableau des matchs
        tbody_matches = table_matches.find_element(By.TAG_NAME, "tbody")
        rows_matches = tbody_matches.find_elements(By.TAG_NAME, "tr")

        for row in rows_matches:
            date = row.find_element(By.TAG_NAME, "th").text
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 17:  # vérifier qu’il y a assez de colonnes
                match_data = {
                    "Date": date,
                    "Time": cols[0].text,
                    "Comp": cols[1].text,
                    "Round": cols[2].text,
                    "Day": cols[3].text,
                    "Venue": cols[4].text,
                    "Result": cols[5].text,
                    "GF": cols[6].text,
                    "GA": cols[7].text,
                    "Opponent": cols[8].text,
                    "xG": cols[9].text,
                    "xGA": cols[10].text,
                    "Poss": cols[11].text,
                    "Attendance": cols[12].text,
                    "Captain": cols[13].text,
                    "Formation": cols[14].text,
                    "Opp Formation": cols[15].text,
                    "Referee": cols[16].text
                }
                all_matches.append(match_data)

        if all_matches:
            df_matches = pd.DataFrame(all_matches)
            file_path_matches = os.path.join(team_dir, f"{team_name.replace(' ', '_')}_matches.csv")
            df_matches.to_csv(file_path_matches, index=False, encoding="utf-8-sig")
    except Exception as e:
        print(f"⚠️ Tableau matchs introuvable pour {team_name} : {e}")

    time.sleep(2)

driver.quit()
print("✅ Données des joueurs et des matchs enregistrées dans les dossiers des équipes")


📊 Équipe: Liverpool
📊 Équipe: Arsenal
📊 Équipe: Manchester City
📊 Équipe: Chelsea
📊 Équipe: Newcastle Utd
📊 Équipe: Aston Villa
📊 Équipe: Nott'ham Forest
📊 Équipe: Brighton
📊 Équipe: Bournemouth
📊 Équipe: Brentford
📊 Équipe: Fulham
📊 Équipe: Crystal Palace
📊 Équipe: Everton
📊 Équipe: West Ham
📊 Équipe: Manchester Utd
📊 Équipe: Wolves
📊 Équipe: Tottenham
📊 Équipe: Leicester City
📊 Équipe: Ipswich Town
📊 Équipe: Southampton
✅ Données des joueurs et des matchs enregistrées dans les dossiers des équipes
